In [1]:
import math
import numpy as np
from PIL import Image

In [4]:
image = Image.open(r'canvas.png')

ImageW, ImageH = image.size
dist = 10
WindowW = 10
WindowH = 10
BACKGROUND_COLOR = np.array([0, 0, 0])

class sphere():
    center = np.array([0, 0, 3])
    radius = 1
    color = np.array([0, 255, 255])


def CanvasToView(x,y):
    return (x*WindowW/ImageW, -y*WindowH/ImageH, dist)

def IntersectRaySphere(O, d, sphere):
    center = sphere.center
    radius = sphere.radius
    oc = O - center

    k1 = np.sum(np.dot(d, d))
    k2 = 2*np.sum(np.dot(oc, d))
    k3 = np.sum(np.dot(oc, oc)) - radius*radius

    discriminant = k2*k2 - 4*k1*k3
    if discriminant < 0:
        return math.inf, math.inf

    t1 = (-k2 + math.sqrt(discriminant)) / (2*k1)
    t2 = (-k2 - math.sqrt(discriminant)) / (2*k1)
    return t1, t2

def TraceRay(O, d, tmin, tmax):
    closest_t = math.inf
    closest_sphere = None
    
    
    #for sphere in scene.Spheres
    t1, t2 = IntersectRaySphere(O, d, sphere)
    if t1 >= tmin and t1 <= tmax and t1 < closest_t:
        closest_t = t1
        closest_sphere = sphere
    if t2 >= tmin and t2 <= tmax and t2 < closest_t:
        closest_t = t2
        closest_sphere = sphere
        
    if closest_sphere == None:
        return BACKGROUND_COLOR
    return closest_sphere.color

O = np.array([0,0,0])

for x in range(-300, 300):
    xx = x + 300
    for y in range(-300, 300):
        yy = y + 300
        color = TraceRay(O, CanvasToView(x, y), 0, math.inf)
        image.putpixel((xx, yy), tuple(int(c) for c in color))
image.show()